In [ ]:
graph_path = "ldbc"
graph_oid = 16714

label_where_key = {
    "forum":"title",
    "organisation":"name",
    "person":"firstname",
    "place":"name",
    "tag":"name",
    "tagclass":"name"
}


In [3]:
import torch
import torch_npu  
print(torch_npu.npu.is_available())

/home/liuyang/miniconda3/envs/pytorch21/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


/home/liuyang/miniconda3/envs/pytorch21/lib/python3.9/site-packages/torch_npu/utils/path_manager.py:82: UserWarning: Warning: The /usr/local/Ascend/ascend-toolkit/latest owner does not match the current user.
  warnings.warn(f"Warning: The {path} owner does not match the current user.")
/home/liuyang/miniconda3/envs/pytorch21/lib/python3.9/site-packages/torch_npu/utils/path_manager.py:82: UserWarning: Warning: The /usr/local/Ascend/ascend-toolkit/8.0.RC1.alpha001/aarch64-linux/ascend_toolkit_install.info owner does not match the current user.
  warnings.warn(f"Warning: The {path} owner does not match the current user.")


In [4]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import DirectoryLoader
model_name = "/home/work/liuytest/demo/maidalun1020/bce-embedding-base_v1"
model_kwargs = {'device': 'npu:0'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
 ## 将name打成向量
embeddings = model.embed_documents(
    [
        "entity_name"
    ]
)
embedding = embeddings[0]
embedding
print(len(embedding))
## pg_con.execute(f'INSERT INTO zyykg_data_{node} (embedding,label, value_name) VALUES (%s,%s,%s)', (embedding,label,entity_name))

/home/liuyang/miniconda3/envs/pytorch21/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
[W compiler_depend.ts:623] Warning: expandable_segments currently defaults to false. You can enable this feature by `export PYTORCH_NPU_ALLOC_CONF = expandable_segments:True`. (function operator())


768


In [ ]:
## 将所有的ldbc schema下边的数据全部添加一列 设置为 embedding类型
import psycopg

pg_con = psycopg.connect(
            dbname="postgres",
            host="localhost",
            user="omm",
            password="ZAQ1234@esz",
            port=5432)
     
       
cur = pg_con.cursor()
pg_con.autocommit=True     
cur.execute(f"select labname from ag_label where graphid = {graph_oid} and labkind = 'v' and labname !='ag_vertex'");
rows = cur.fetchall()
for row in rows:
    labelname = str(row[0])
    if labelname in label_where_key:
        print(f"ALTER table {graph_path}.{row[0]} add COLUMN feature_emd vector(768)")
        cur.execute(f"ALTER table {graph_path}.{row[0]} add COLUMN feature_emd vector(768)")
pg_con.close()

ALTER table ldbc.person add COLUMN feature_emd vector(768)


In [5]:
## 将所有的ldbc schema下边的数据全部添加一列 设置为 embedding类型

import psycopg
import json


 
pg_con = psycopg.connect(
            dbname="postgres",
            host="localhost",
            user="omm",
            password="ZAQ1234@esz",
            port=5432)
     
       
cur = pg_con.cursor()
pg_con.autocommit=True
   
cur.execute(f"select labname from ag_label where graphid = {graph_oid} and labkind = 'v' and labname !='ag_vertex'");
rows = cur.fetchall()
for row in rows:
    print('name=' + str(row[0]))
    labelname = str(row[0])
    if labelname in label_where_key:
        print(f"select * {graph_path}.{labelname} limit 1")
        cur.execute(f"select * from {graph_path}.{labelname}")
        # 获取列名
        column_names = [desc[0] for desc in cur.description]
        print(" | ".join(column_names))  # 打印列名
        # 获取查询结果
        results = cur.fetchall()
        where_key = label_where_key[labelname]
        # 打印结果
        for row in results:
            graphid = row[0] 
            properties = json.loads(json.dumps(row[1]))
            entity_value = properties[where_key]
            ## 将name打成向量
            embeddings = model.embed_documents(
                [
                    entity_value
                ]
            )
            embedding = embeddings[0]
            cur.execute(f"update   {graph_path}.{labelname}  set feature_emd = %s where id = %s",(embedding,graphid))
pg_con.close()

name=forum
name=message
name=post
name=comment
name=organisation
name=person
select * ldbc.person limit 1
id | properties | feature_emd
name=place
select * ldbc.place limit 1
id | properties | feature_emd
name=tag
select * ldbc.tag limit 1
id | properties | feature_emd
name=tagclass
select * ldbc.tagclass limit 1
id | properties | feature_emd
